# import Libraries

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
from classes import SentimentDataset
import torch.optim as optim
load_path = 'sentiment_data_loader.pth' # Đảm bảo đường dẫn này chính xác
loaded_data = torch.load(load_path)
batch_size = loaded_data.get('batch_size', 32)
train_loader = loaded_data.get('train_loader')
test_loader = loaded_data.get('test_loader')
vocab = loaded_data.get('vocab')
vocab_size = loaded_data.get('vocab_size')

In [27]:
vocab_size

5000

# Build Model

In [28]:
# Phul luc B: model.py
import torch.nn as nn
import torchtext.vocab as tvocab
import numpy as np

# --- Helper function to load GloVe embeddings ---
def load_glove_embeddings(glove_path, vocab, embedding_dim):
    """
    Loads GloVe embeddings for words found in the vocabulary.

    Args:
        glove_path (str): Name of the GloVe vectors (e.g., 'glove.6B.100d').
                          Make sure you have downloaded these or torchtext can download them.
        vocab (dict): The vocabulary mapping words to indices.
        embedding_dim (int): The dimension of the GloVe embeddings.

    Returns:
        torch.Tensor: The embedding matrix.
    """
    print(f"Loading GloVe vectors: {glove_path}...")
    # Tải GloVe vectors sử dụng torchtext
    # Lần đầu chạy có thể mất thời gian để tải file GloVe
    try:
        glove = tvocab.GloVe(name=glove_path.split('.')[1], # e.g., '6B'
                             dim=embedding_dim,            # e.g., 100
                             cache='.vector_cache')        # Thư mục lưu cache
        print("GloVe vectors loaded successfully.")
    except Exception as e:
        print(f"Error loading GloVe vectors: {e}")
        print("Please ensure the GloVe files are available or can be downloaded.")
        print("You might need to install torchtext: pip install torchtext")
        # Hoặc tải thủ công từ: https://nlp.stanford.edu/projects/glove/
        # và giải nén vào thư mục .vector_cache
        raise e # Dừng chương trình nếu không tải được GloVe

    vocab_size = len(vocab)
    # Khởi tạo ma trận embedding với giá trị ngẫu nhiên nhỏ
    embeddings = np.random.uniform(-0.25, 0.25, (vocab_size, embedding_dim))
    embeddings[vocab['<PAD>']] = np.zeros(embedding_dim) # Vector 0 cho PAD

    # Điền vào ma trận embedding bằng vector GloVe nếu từ có trong GloVe
    loaded_count = 0
    for word, idx in vocab.items():
        if word in glove.stoi: # stoi: string-to-index mapping trong GloVe object

            embeddings[idx] = glove.vectors[glove.stoi[word]].numpy()
            loaded_count += 1
        # else: để lại giá trị khởi tạo ngẫu nhiên (hoặc có thể gán vector <UNK> nếu muốn)

    print(f"Loaded {loaded_count} vectors from GloVe out of {vocab_size} vocab size.")
    return torch.tensor(embeddings, dtype=torch.float)
# --------------------------------------------------
#pretrained_embeddings = load_glove_embeddings('glove.6B.100d',vocab,100)

In [29]:
class RNNModel(nn.Module):
    def __init__(self,vocab, vocab_size, embedding_dim, hidden_dim, output_dim,
                 pad_idx, pretrained=False, glove_path='glove.6B.100d'): # Thêm vocab và glove_path
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.vocab_size = vocab_size
        self.padding_idx = pad_idx # Lấy index của PAD token

        # --- Khởi tạo embedding layer ---
        if pretrained:
            print("Using pre-trained GloVe embeddings.")
            # Tải trọng số GloVe
            pretrained_embeddings = load_glove_embeddings(glove_path, vocab, embedding_dim)
            # Tạo lớp Embedding từ trọng số đã tải
            self.embedding = nn.Embedding.from_pretrained(
                pretrained_embeddings,
                freeze=False, # Cho phép fine-tuning embedding nếu muốn (False)
                padding_idx=self.padding_idx
            )
        else:
            print("Training embeddings from scratch.")
            # Khởi tạo embedding ngẫu nhiên
            self.embedding = nn.Embedding(
                num_embeddings=vocab_size,
                embedding_dim=embedding_dim,
                padding_idx=self.padding_idx
            )

        # --- Khởi tạo khối RNN layer ---
        # [Sinh viên bổ sung: dùng nn.RNN với batch_first=True]
        self.rnn = nn.RNN(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            bidirectional = True,
            num_layers=1, # Giữ đơn giản với 1 lớp RNN
            batch_first=True, # Quan trọng: input/output có dạng (batch, seq, feature)
        )
        self.dropout = nn.Dropout(0.5)
        # --- Khởi tạo tầng Dense để dự đoán 3 nhãn ---
        # [Sinh viên bổ sung: dùng nn.Linear, nhận hidden state từ RNN]
        self.fc = nn.Linear(
            in_features=hidden_dim, # Input là hidden state cuối cùng của RNN
            out_features=output_dim # Output là số lớp cảm xúc (3)
        )
        # --- Hết phần bổ sung Dense ---

    def forward(self, text):
        # text shape: (batch_size, seq_len)

        # --- Chuyển text thành embedding ---
        # [Sinh viên bổ sung]
        # embedded shape: (batch_size, seq_len, embedding_dim)
        embedded = self.embedding(text)
        # --- Hết phần bổ sung embedding forward ---

        # --- Đưa qua khối RNN để lẩy hidden state cuối ---
        # [Sinh viên bổ sung]
        # output shape: (batch_size, seq_len, hidden_dim)
        # hidden shape: (num_layers, batch_size, hidden_dim) -> (1, batch_size, hidden_dim)
        rnn_output, hidden = self.rnn(embedded)
        #_, (hidden, _) = self.rnn(embedded)
        # Lấy hidden state cuối cùng của lớp RNN duy nhất
        # hidden.squeeze(0) loại bỏ chiều num_layers (vì = 1)
        # last_hidden shape: (batch_size, hidden_dim)
        #last_hidden = hidden.squeeze(0)
        last_hidden = hidden[-1] 
        # --- Hết phần bổ sung RNN forward ---

        # --- Đưa hidden state qua tầng Dense để dự đoán 3 nhãn ---
        # [Sinh viên bổ sung]
        # predictions shape: (batch_size, output_dim)
        last_hidden = self.dropout(last_hidden)  # <-- thêm dòng này

        predictions = self.fc(last_hidden)
        # --- Hết phần bổ sung Dense forward ---

        # [Sinh viên bổ sung: trả về kết quả dự đoán]
        return predictions


# Train Model

In [30]:
# --- Configuration ---
embedding_dim = 100 # Kích thước vector embedding
hidden_dim = 128    # Kích thước lớp ẩn RNN
output_dim = 3
pad_idx = vocab["<PAD>"]

In [31]:
model_glove= RNNModel(vocab,vocab_size, embedding_dim, hidden_dim, output_dim,pad_idx, pretrained=True,glove_path='glove.6B.100d')
model_scratch= RNNModel(vocab,vocab_size, embedding_dim, hidden_dim, output_dim,pad_idx, pretrained=False)


Using pre-trained GloVe embeddings.
Loading GloVe vectors: glove.6B.100d...
GloVe vectors loaded successfully.
Loaded 537 vectors from GloVe out of 5000 vocab size.
Training embeddings from scratch.


In [32]:
len(train_loader)

405

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import json
from sklearn.metrics import accuracy_score, f1_score

def train_and_evaluate_model(model, train_loader, test_loader, model_name="model", num_epochs=100, learning_rate=0.01, save_path="metrics.json"):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    print("\n--- Starting Training ---")
    start_time = time.time()

    metrics = {}

    for epoch in range(num_epochs):
        model.train()
        for batch_sequences, batch_labels in train_loader:
            batch_sequences = batch_sequences.to(device)
            batch_labels = batch_labels.to(device)

            optimizer.zero_grad()
            predictions = model(batch_sequences)
            loss = criterion(predictions, batch_labels)
            loss.backward()
            optimizer.step()

        # --- Validation ---
        model.eval()
        val_preds = []
        val_labels_all = []

        with torch.no_grad():
            for val_sequences, val_labels in test_loader:
                val_sequences = val_sequences.to(device)
                val_labels = val_labels.to(device)

                val_predictions = model(val_sequences)
                preds_class = torch.argmax(val_predictions, dim=1)

                val_preds.extend(preds_class.cpu().numpy())
                val_labels_all.extend(val_labels.cpu().numpy())

        val_accuracy = accuracy_score(val_labels_all, val_preds)
        val_f1 = f1_score(val_labels_all, val_preds, average='weighted')

        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}] - Val Acc: {val_accuracy:.4f}, F1: {val_f1:.4f}')

    # --- Save metrics to JSON ---
    metrics[model_name] = {
        "val_accuracy": round(val_accuracy, 4),
        "val_f1_score": round(val_f1, 4)
    }

    with open(save_path, 'w') as f:
        json.dump(metrics, f, indent=4)

    print(f"\n✅ Metrics saved to {save_path}")

for (model,model_name) in [(model_glove,"Model_pretrained"),(model_scratch,"Model_scratch")]:
    train_and_evaluate_model(model,train_loader,test_loader,model_name=model_name,num_epochs=50)


Using device: cpu

--- Starting Training ---
Epoch [1/50] - Val Acc: 0.5781, F1: 0.5173
Epoch [10/50] - Val Acc: 0.7357, F1: 0.7116
Epoch [20/50] - Val Acc: 0.7750, F1: 0.7548
Epoch [30/50] - Val Acc: 0.7972, F1: 0.7787
Epoch [40/50] - Val Acc: 0.8102, F1: 0.7940
Epoch [50/50] - Val Acc: 0.8142, F1: 0.8001

✅ Metrics saved to metrics.json
Using device: cpu

--- Starting Training ---
Epoch [1/50] - Val Acc: 0.6903, F1: 0.6754
Epoch [10/50] - Val Acc: 0.7669, F1: 0.7502
Epoch [20/50] - Val Acc: 0.7747, F1: 0.7568
Epoch [30/50] - Val Acc: 0.8040, F1: 0.7880
Epoch [40/50] - Val Acc: 0.7929, F1: 0.7768
Epoch [50/50] - Val Acc: 0.8006, F1: 0.7871

✅ Metrics saved to metrics.json


In [34]:
# import time
# # --- Xác định thiết bị (CPU hoặc GPU nếu có) ---
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Using device: {device}")
# learning_rate = 0.01 # Tốc độ học cho SGD
# num_epochs = 100
# # Khởi tạo mô hình
# pad_idx = vocab['<PAD>']
# # model = RNNModel(vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx)
# #model = model.to(device) # Chuyển model lên device
# model = model_glove.to(device)
# # --- Bước 3: Định nghĩa Loss và Optimizer ---
# criterion = nn.CrossEntropyLoss() # Phù hợp cho bài toán phân loại đa lớp
# optimizer = optim.SGD(model.parameters(), lr=learning_rate) # Sử dụng SGD theo yêu cầu

# # --- Bước 4: Huấn luyện mô hình ---
# print("\n--- Starting Training ---")
# start_time = time.time()

# from sklearn.metrics import f1_score

# for epoch in range(num_epochs):
#     model.train()
#     epoch_loss = 0
#     epoch_correct = 0
#     epoch_total = 0
#     all_preds = []
#     all_labels = []

#     for batch_sequences, batch_labels in train_loader:
#         batch_sequences = batch_sequences.to(device)
#         batch_labels = batch_labels.to(device)
#         optimizer.zero_grad()

#         predictions = model(batch_sequences)
#         loss = criterion(predictions, batch_labels)
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()
#         preds_class = torch.argmax(predictions, dim=1)
#         epoch_correct += (preds_class == batch_labels).sum().item()
#         epoch_total += batch_labels.size(0)

#         all_preds.extend(preds_class.cpu().numpy())
#         all_labels.extend(batch_labels.cpu().numpy())

#     avg_epoch_loss = epoch_loss / len(train_loader)
#     train_accuracy = epoch_correct / epoch_total
#     train_f1 = f1_score(all_labels, all_preds, average='weighted')  # or 'macro' if labels are balanced

#     # --- Validation ---
#     model.eval()
#     val_loss = 0
#     val_correct = 0
#     val_total = 0
#     val_preds = []
#     val_labels_all = []

#     with torch.no_grad():
#         for val_sequences, val_labels in test_loader:
#             val_sequences = val_sequences.to(device)
#             val_labels = val_labels.to(device)

#             val_predictions = model(val_sequences)
#             loss = criterion(val_predictions, val_labels)

#             val_loss += loss.item()
#             preds_class = torch.argmax(val_predictions, dim=1)
#             val_correct += (preds_class == val_labels).sum().item()
#             val_total += val_labels.size(0)

#             val_preds.extend(preds_class.cpu().numpy())
#             val_labels_all.extend(val_labels.cpu().numpy())

#     avg_val_loss = val_loss / len(test_loader)
#     val_accuracy = val_correct / val_total
#     val_f1 = f1_score(val_labels_all, val_preds, average='weighted')  # or 'macro'

#     if (epoch + 1) % 10 == 0 or epoch == 0:
#         print(f'Epoch [{epoch+1}/{num_epochs}], '
#               f'Train Loss: {avg_epoch_loss:.4f}, Train Acc: {train_accuracy:.4f}, F1: {train_f1:.4f}, '
#               f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}, F1: {val_f1:.4f}')
